# Monk's Problem

Import our own neural network code aswell as numpy and pandas for data handling and loading.

In [1]:
import numpy as np
import pandas as pd
from network import *
from train_utils import *
from tqdm import tqdm
import matplotlib.pyplot as plt

Read the data into pandas dataframes, remove empty first column, remove ids, split into variables and class labels.

In [2]:
X1_df = pd.read_csv("data/monks/monks-1.train", sep=" ", header=None)
XT1_df = pd.read_csv("data/monks/monks-1.test", sep=" ", header=None)
X2_df = pd.read_csv("data/monks/monks-2.train", sep=" ", header=None)
XT2_df = pd.read_csv("data/monks/monks-2.test", sep=" ", header=None)
X3_df = pd.read_csv("data/monks/monks-3.train", sep=" ", header=None)
XT3_df = pd.read_csv("data/monks/monks-3.test", sep=" ", header=None)

Y1_df = X1_df.pop(1)
X1_df = X1_df.drop(columns=[0, 8])
YT1_df = XT1_df.pop(1)
XT1_df = XT1_df.drop(columns=[0, 8])
Y2_df = X2_df.pop(1)
X2_df = X2_df.drop(columns=[0, 8])
YT2_df = XT2_df.pop(1)
XT2_df = XT2_df.drop(columns=[0, 8])
Y3_df = X3_df.pop(1)
X3_df = X3_df.drop(columns=[0, 8])
YT3_df = XT3_df.pop(1)
XT3_df = XT3_df.drop(columns=[0, 8])

print(X1_df.shape)
print(X1_df.head())
print(Y1_df.shape)
print(Y1_df.head())

(124, 6)
   2  3  4  5  6  7
0  1  1  1  1  3  1
1  1  1  1  1  3  2
2  1  1  1  3  2  1
3  1  1  1  3  3  2
4  1  1  2  1  2  1
(124,)
0    1
1    1
2    1
3    1
4    1
Name: 1, dtype: int64


Apply one-hot encoding to each variable and convert to numpy arrays.

In [3]:
def one_hot_encode_feature(df, feature_to_encode):
    dummies = pd.get_dummies(df[feature_to_encode], dtype="int32", prefix=feature_to_encode)
    result_df = pd.concat([df, dummies], axis=1)
    return result_df.drop(columns=feature_to_encode)

for feature in X1_df.columns:
    X1_df = one_hot_encode_feature(X1_df, feature)
    XT1_df = one_hot_encode_feature(XT1_df, feature)
    X2_df = one_hot_encode_feature(X2_df, feature)
    XT2_df = one_hot_encode_feature(XT2_df, feature)
    X3_df = one_hot_encode_feature(X3_df, feature)
    XT3_df = one_hot_encode_feature(XT3_df, feature)

X1 = X1_df.to_numpy()
Y1 = Y1_df.to_numpy().reshape(-1, 1)
XT1 = XT1_df.to_numpy()
YT1 = YT1_df.to_numpy().reshape(-1, 1)
X2 = X2_df.to_numpy()
Y2 = Y2_df.to_numpy().reshape(-1, 1)
XT2 = XT2_df.to_numpy()
YT2 = YT2_df.to_numpy().reshape(-1, 1)
X3 = X3_df.to_numpy()
Y3 = Y3_df.to_numpy().reshape(-1, 1)
XT3 = XT3_df.to_numpy()
YT3 = YT3_df.to_numpy().reshape(-1, 1)
print(X1.shape)
print(Y1.shape)
print(X1)

(124, 17)
(124, 1)
[[1 0 0 ... 0 1 0]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]
 ...
 [0 0 1 ... 0 0 1]
 [0 0 1 ... 0 0 1]
 [0 0 1 ... 1 0 1]]


## Initial Training
We first train with hould-out validation and hold-out test to see if training works at all.

Split data into train and validation sets, using 80% of the data for train and 20% for validation. The test set is already given as part of the dataset.

In [4]:
def split_data(X, Y, train_fraction=0.8, shuffle=True):
    n = X.shape[0]
    indices = np.arange(n)
    if shuffle:
        np.random.shuffle(indices)

    train_size = int(n * train_fraction)
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    X_test = X[test_indices]
    Y_test = Y[test_indices]
    return X_train, Y_train, X_test, Y_test

XTr1, YTr1, XVl1, YVl1 = split_data(X1, Y1)
XTr2, YTr2, XVl2, YVl2 = split_data(X2, Y2)
XTr3, YTr3, XVl3, YVl3 = split_data(X3, Y3)

print(XTr1.shape, YTr1.shape, XVl1.shape, YVl1.shape)

(99, 17) (99, 1) (25, 17) (25, 1)


Define datasets and dataloaders.

In [5]:
XTr1_dl = DataLoader(Dataset(XTr1, YTr1), batch_size=8, shuffle=True)
XTr2_dl = DataLoader(Dataset(XTr2, YTr2), batch_size=8, shuffle=True)
XTr3_dl = DataLoader(Dataset(XTr3, YTr3), batch_size=8, shuffle=True)
XVl1_dl = DataLoader(Dataset(XVl1, YVl1), batch_size=8, shuffle=False)
XVl2_dl = DataLoader(Dataset(XVl2, YVl2), batch_size=8, shuffle=False)
XVl3_dl = DataLoader(Dataset(XVl3, YVl3), batch_size=8, shuffle=False)
XT1_dl = DataLoader(Dataset(XT1, YT1), batch_size=8, shuffle=False)
XT2_dl = DataLoader(Dataset(XT2, YT2), batch_size=8, shuffle=False)
XT3_dl = DataLoader(Dataset(XT3, YT3), batch_size=8, shuffle=False)

Train a small model to see if training works at all.

In [6]:
model = Model(
    LinearLayer(17, 3),
    ReLU(),
    LinearLayer(3, 1),
    Sigmoid(),
)
loss_fn = BCELoss()
optimizer = AdamWOptimizer(model, learning_rate=0.01, weight_decay=0.01)



for epoch in range(25):
    # Train
    train_total_n = 0
    train_losses = 0.0
    train_correct_n = 0
    for x_batch, y_batch in XTr1_dl:
        y_pred = model.forward(x_batch)
        loss = loss_fn.forward(y_pred, y_batch)
        grad_loss = loss_fn.backward()
        model.backward(grad_loss)
        optimizer.step()
        train_total_n += y_batch.shape[0]
        train_losses += loss * y_batch.shape[0]
        y_hat = (y_pred >= 0.5).astype(int)
        train_correct_n += np.sum(y_hat == y_batch)

    # Validate
    val_total_n = 0
    val_losses = 0.0
    val_correct_n = 0
    for x_batch, y_batch in XVl1_dl:
        y_pred = model.forward(x_batch)
        loss = loss_fn.forward(y_pred, y_batch)
        val_total_n += y_batch.shape[0]
        val_losses += loss * y_batch.shape[0]
        y_hat = (y_pred >= 0.5).astype(int)
        val_correct_n += np.sum(y_hat == y_batch)

    print(f"Epoch {epoch+1:03d} | Train Loss: {train_losses / train_total_n:.4f} | Train Acc: {train_correct_n / train_total_n:.4f} | Val Loss: {val_losses / val_total_n:.4f} | Val Acc: {val_correct_n / val_total_n:.4f}")

Epoch 001 | Train Loss: 0.7875 | Train Acc: 0.4343 | Val Loss: 0.7253 | Val Acc: 0.3600
Epoch 002 | Train Loss: 0.6836 | Train Acc: 0.5253 | Val Loss: 0.7049 | Val Acc: 0.4400
Epoch 003 | Train Loss: 0.6119 | Train Acc: 0.6162 | Val Loss: 0.6910 | Val Acc: 0.3600
Epoch 004 | Train Loss: 0.5630 | Train Acc: 0.6465 | Val Loss: 0.6638 | Val Acc: 0.4800
Epoch 005 | Train Loss: 0.5213 | Train Acc: 0.6465 | Val Loss: 0.6293 | Val Acc: 0.5200
Epoch 006 | Train Loss: 0.4788 | Train Acc: 0.7374 | Val Loss: 0.6039 | Val Acc: 0.7600
Epoch 007 | Train Loss: 0.4500 | Train Acc: 0.8081 | Val Loss: 0.5906 | Val Acc: 0.7200
Epoch 008 | Train Loss: 0.4220 | Train Acc: 0.8384 | Val Loss: 0.5780 | Val Acc: 0.7200
Epoch 009 | Train Loss: 0.3904 | Train Acc: 0.8384 | Val Loss: 0.5733 | Val Acc: 0.7200
Epoch 010 | Train Loss: 0.3655 | Train Acc: 0.8586 | Val Loss: 0.5771 | Val Acc: 0.7200
Epoch 011 | Train Loss: 0.3467 | Train Acc: 0.8586 | Val Loss: 0.5768 | Val Acc: 0.7200
Epoch 012 | Train Loss: 0.3384 |

In [69]:
# Test
total_n = 0
test_losses = 0.0
correct_n = 0
for x_batch, y_batch in XT1_dl:
    y_pred = model.forward(x_batch)
    loss = loss_fn.forward(y_pred, y_batch)
    total_n += y_batch.shape[0]
    test_losses += loss * y_batch.shape[0]
    y_hat = (y_pred >= 0.5).astype(int)
    correct_n += np.sum(y_hat == y_batch)

print(f"Test Loss: {test_losses / total_n:.4f} | Test Acc: {correct_n / total_n:.4f}")

Test Loss: 0.4795 | Test Acc: 0.7824


## Cross Validation + Grid Search
Now that we have a basic model that is able to learn, we do a cross validation with grid search and hold out the test set each time (because the test set was already separated as part of the Monk's problem).

The relevant functions are in `train_utils.py`.


In [ ]:
parameters = dict(
    learning_rates = [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    weight_decays = [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    input_size = 17,
    hidden_units_layer_1 = [2, 3, 4, 5, 6, 7],
    hidden_units_layer_2 = [0, 2, 3, 4, 5, 6, 7],
    hidden_units_layer_3 = [0],
    output_size = 1,
    batch_sizes = [8],
    activations_hidden_layer = [ReLU],
    activations_output_layer = [Sigmoid],
    loss_functions = [BCELoss],
    optimizers = [AdamWOptimizer],
)

cfgs = generate_param_cfgs(parameters)
print(len(cfgs))
print(cfgs[0])

2268
{'learning_rate': 0.0005, 'weight_decay': 0.0005, 'input_size': 17, 'hidden_units_layer_1': 2, 'hidden_units_layer_2': 0, 'hidden_units_layer_3': 0, 'batch_size': 8, 'output_size': 1, 'activation_hidden_layer': <class 'network.relu.ReLU'>, 'activation_output_layer': <class 'network.sigmoid.Sigmoid'>, 'loss_function': <class 'network.bce.BCELoss'>, 'optimizer': <class 'network.adam.AdamWOptimizer'>}


First Monk 1, then 2 and 3.

In [9]:
results = grid_search_cross_validate(X1, Y1, cfgs, final_metric_fn=accuracy)
best_cfg, best_result, epochs = max(results, key=lambda x: x[1])
print("Best config:", best_cfg)
print(f"Best validation acc: {best_result:.4f}")
print(f"Used epochs: {epochs}")

100%|██████████| 2268/2268 [06:57<00:00,  5.43it/s]

Best config: {'learning_rate': 0.05, 'weight_decay': 0.1, 'input_size': 17, 'hidden_units_layer_1': 6, 'hidden_units_layer_2': 6, 'hidden_units_layer_3': 0, 'batch_size': 8, 'output_size': 1, 'activation_hidden_layer': <class 'network.relu.ReLU'>, 'activation_output_layer': <class 'network.sigmoid.Sigmoid'>, 'loss_function': <class 'network.bce.BCELoss'>, 'optimizer': <class 'network.adam.AdamWOptimizer'>}
Best validation acc: 0.8030
Used epochs: 33


Retrain best found parameters on train + validation set, then assess performance on test set. Repeat n times and average results, because dataset is very small and initialization matters.

In [94]:
X1_dl = DataLoader(Dataset(X1, Y1), batch_size=best_cfg['batch_size'], shuffle=False)

test_losses = []
test_accs = []
for _ in range(100):
    best_model = create_model(best_cfg)
    loss_fn = best_cfg['loss_function']()
    optimizer = best_cfg['optimizer'](best_model, learning_rate=best_cfg['learning_rate'], weight_decay=best_cfg['weight_decay'])
    best_model, _, _ = train_model(best_model, loss_fn, optimizer, X1_dl, None, accuracy, max_epochs=epochs)
    test_loss, test_acc = epoch_run(best_model, loss_fn, None, XT1_dl, accuracy, update_weights=False)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

test_loss = np.mean(test_losses)
test_acc = np.mean(test_accs)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

Test Loss: 0.4262 | Test Acc: 0.7733
